# Analyse exploratoire

Visualisation de la consommation pour plusieurs horizons (global, semaine, journée, mois, hiver, été) et inspection visuelle des patterns (pics matin/soir, effet weekend, saisonnalité).

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

from src.data_processing import load_uci_household, preprocess_household_hourly

plt.style.use("seaborn-v0_8-darkgrid")
pd.options.display.max_columns = 20
pd.options.display.width = 120

ModuleNotFoundError: No module named 'src'

In [ ]:
# Chargement et agrégation horaire
DATA_PATH = Path("data/consumption_data.csv")
if not DATA_PATH.exists():
    DATA_PATH = Path("../data/consumption_data.csv")

raw = load_uci_household(DATA_PATH)
hourly = preprocess_household_hourly(raw)

cons = hourly["Global_active_power"].rename("load_kw")
cons.head()

## Courbe globale
Moyenne quotidienne pour visualiser l'évolution globale.

In [ ]:
global_daily = cons.resample("D").mean()
fig, ax = plt.subplots(figsize=(12, 4))
global_daily.plot(ax=ax, color="tab:blue", linewidth=1)
ax.set_title("Consommation moyenne quotidienne (kW)")
ax.set_xlabel("")
ax.set_ylabel("kW")
plt.tight_layout()
plt.show()

## Fenêtres temporelles
- Semaine
- Journée
- Mois
- Hiver
- Été

In [ ]:
def plot_window(series: pd.Series, start: str, end: str, title: str):
    window = series.loc[start:end]
    fig, ax = plt.subplots(figsize=(12, 4))
    window.plot(ax=ax, color="tab:orange", linewidth=1)
    ax.set_title(f"{title} ({start} → {end})")
    ax.set_xlabel("")
    ax.set_ylabel("kW")
    plt.tight_layout()
    plt.show()


plot_window(cons, "2010-01-04", "2010-01-10", "Semaine (janvier 2010)")
plot_window(cons, "2010-01-05", "2010-01-05 23:59:59", "Journée (5 janvier 2010)")
plot_window(cons, "2010-01-01", "2010-01-31", "Mois (janvier 2010)")
plot_window(cons, "2009-12-01", "2010-02-28", "Hiver (déc 2009 - fév 2010)")
plot_window(cons, "2010-07-01", "2010-08-31", "Été (juil-août 2010)")

## Patterns horaires et weekend
Visualisation des pics matin/soir et de l'effet weekend.

In [ ]:
# Profil moyen par heure (tous jours)
hod = cons.groupby(cons.index.hour).mean()
fig, ax = plt.subplots(figsize=(10, 4))
hod.plot(kind="bar", ax=ax, color="tab:blue")
ax.set_title("Profil moyen par heure")
ax.set_xlabel("Heure du jour")
ax.set_ylabel("kW")
plt.tight_layout()
plt.show()

# Profil moyen par heure séparant semaine/weekend
tmp = cons.to_frame()
tmp["hour"] = tmp.index.hour
tmp["is_weekend"] = tmp.index.weekday >= 5
pivot = tmp.groupby(["is_weekend", "hour"]).mean().unstack(0)["load_kw"]
pivot.columns = ["Semaine", "Weekend"]
fig, ax = plt.subplots(figsize=(10, 4))
pivot.plot(ax=ax)
ax.set_title("Profil horaire moyen : semaine vs weekend")
ax.set_xlabel("Heure du jour")
ax.set_ylabel("kW")
plt.tight_layout()
plt.show()

## Saisonnalité
Profil mensuel pour repérer la saisonnalité.

In [ ]:
monthly = cons.resample("M").mean()
fig, ax = plt.subplots(figsize=(12, 4))
monthly.plot(ax=ax, color="tab:green", linewidth=1)
ax.set_title("Consommation moyenne mensuelle (kW)")
ax.set_xlabel("")
ax.set_ylabel("kW")
plt.tight_layout()
plt.show()